In [ ]:
import sqlite3

# Definição da classe Produto
class Produto:
    def __init__(self, nome, preco):
        self.nome = nome
        self.preco = preco

# Função para inserir produto no banco
def inserir_produto(conn, produto):
    cursor = conn.cursor()
    cursor.execute('INSERT INTO produtos (nome, preco) VALUES (?, ?)', (produto.nome, produto.preco))
    conn.commit()
    return cursor.lastrowid

# Função para inserir venda no banco
def inserir_venda(conn, produto_id, vendas):
    cursor = conn.cursor()
    cursor.execute('INSERT INTO vendas (produto_id, vendas) VALUES (?, ?)', (produto_id, vendas))
    conn.commit()

# Teste de integração para inserir venda
def test_integracao_inserir_venda():
    conn = sqlite3.connect(':memory:')  # Banco em memória

    cursor = conn.cursor()
    # Criar tabela produtos
    cursor.execute('''CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        preco REAL
    )''')

    # Criar tabela vendas
    cursor.execute('''CREATE TABLE IF NOT EXISTS vendas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        produto_id INTEGER,
        vendas INTEGER,
        FOREIGN KEY (produto_id) REFERENCES produtos (id)
    )''')
    conn.commit()

    # Criar um produto e inserir
    produto = Produto("Teclado", 75.00)
    produto_id = inserir_produto(conn, produto)

    # Inserir venda associada ao produto
    inserir_venda(conn, produto_id, 50)

    # Verificar se a venda foi inserida corretamente
    cursor.execute('SELECT produto_id, vendas FROM vendas WHERE produto_id = ?', (produto_id,))
    resultado = cursor.fetchone()

    assert resultado is not None, "Nenhuma venda encontrada."
    assert resultado[0] == produto_id, f"Esperado produto_id {produto_id}, mas veio {resultado[0]}"
    assert resultado[1] == 50, f"Esperado vendas 50, mas veio {resultado[1]}"

    conn.close()

# Executa o teste no Colab
test_integracao_inserir_venda()
print("✅ Teste de integração de venda passou!")